In [203]:
import os
import pandas as pd
import glob
import string
import nltk
import re


In [212]:
import os
import pandas as pd
import glob
import re


class Utils():
    root_directory = os.chdir('c:/Users/iZiPC.by/notebooks/parser/')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    for i in trash: 
        list_of_files.remove(i)


list_of_files = Utils.list_of_files


plt_ = []
wbc_ = []
hgb_ = []
crea_ = []
ldh_ = []
sex_ = []
crp_ = []
tr = []
bir = []
ad = []
dis = []
ID_ = []
dgs_ = []
rf_ = []
alt_ = []
ast_ = []
pct_ = []

def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'
            
            
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('лечение', line):  # Does the same thing as "if 'hello' in line:"
        return(line.replace('Проведенное лечение:', \
                            '').replace('ЛФК', '').replace('ФТЛ', ''))
    

def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'


pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:',
                     "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_ = 0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+40]    
  

def crp(file_):
  """returns all crp values"""
  pattern = re.sub(r'[ЦСC]РБ', 'С-реактивныйбелок', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)\
                         (\d*.\d+|\d+)')
  pattern_2 = pattern_1.findall(''.join(pattern.split()))
  try:
    return pattern_2
  except:
    return 'None'


def gender(file_):
  file_ = file_.title()
  file_ = ''.join(file_.split())
  file_ = re.sub(r'Диагноз.*', '', file_)
  patt1 = re.compile(r'(?<=Ф.И.О:)[А-Я]\w{,20}[А-Я]\w{,20}(вич)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'male'
    else:
      return 'female'
  except:
      pass
  

def ldh(file_):
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass



def cre(file_):
  """Returns all creatinine values"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  patt1 = re.compile(r'(?<=cre)\d{,3}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(''.join(patt1.split()))
  try:
    if all_hgb:
      return all_hgb
    else:
      return 'None'
  except:
    pass


def wbc(file_):
  """returns list of str with level of wbc"""
  file_ = file_.lower()
  patt1 = re.sub(r'[():]', '', file_)
  patt1 = re.sub(r'10(\*|[еe])9', '', patt1)
  patt1 = re.sub(r'\Dбщийанализкрови', 'оак', patt1) 
  patt2 = re.sub(r'(?<=оак)\d\d.\d\d.\d{2,4}|(?<=оак)\d\d.\d\d', '', patt1)
  patt3 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*.\d+|\d+)')
  all_wbc = patt3.findall(''.join(patt2.split()))
  try:
    if all_wbc:
      return all_wbc
    else:
      return 'None'
  except:
    pass


def plt(file_):
  """returns list of str with level of plt"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  patt2 = re.compile(r'(?:\wромбоцит\w|тр)')
  patt1 = re.sub(patt2, '', file_)
  patt1 = ''.join(patt1.split())
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(patt1)
  try:
    if all_plt:
      return all_plt
    else:
      return 'None'
  except:
    pass


def rf(file_):
  file_= ''.join(file_.split())  
  file_ = re.sub(r'[а-яА-ЯёЁ]ДН[^Шш0-9]', 'ДН'.lower(), file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{3}|[Шш]', '3', file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{2}|1-2', '2', file_)
  file_ = re.sub(r'(?<=ДН)[оoОO]', '0', file_)
  patt1 = re.compile(r'(?<=ДН)\d')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'  
  except:
      pass

def alt(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r',', '.', file_)
  file_ = re.sub(r'\wланинаминотрансфераз\w|алат', 'алт', file_)
  patt1 = re.compile(r'(?<=алт)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def ast(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r',', '.', file_)
  file_ = re.sub(r'\wспартатаминотрансфераз\w|асат', 'аст', file_)
  patt1 = re.compile(r'(?<=аст)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def pct(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[–-]|[():<>]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r',', '.', file_)
  file_ = re.sub(r'(\wрокальцитони\w|прокальцитон|прокальцит|прокальц)(?![а-яА-Я,])', 'pct', file_)
  patt1 = re.compile(r'(?<=pct)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass

In [213]:
for file in list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(''.join(file_.split())))
                sex_.append(gender(''.join(file_.split())))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
                rf_.append(rf(file_))
                alt_.append(alt(file_))
                ast_.append(ast(file_))
                pct_.append(pct(file_))
    except:
        print('stp')

In [214]:
for i in pct_:
    if i != "None":
        print(i)

['0.07']
['0.10']
['0.07']
['0.07']
['0.37', '0.27', '0.1', '0.08']
['0.14']
['.07']
['0.98']


In [215]:
rfl = []
for o in rf_:
    if o:
        rfl.append(max(o))
    else:
        rfl.append('None')

In [216]:
diagnosis_cleaned =  []
for diagnos in dgs_:
    try:
        diagnosis_cleaned.append(' '.join(diagnos))
    except:
        diagnosis_cleaned.append('None')

In [217]:
len(diagnosis_cleaned)

2002

In [218]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [219]:
list_wo_punct = []
for sent in diagnosis_cleaned:
    try:
        sent = remove_punctuation(sent.lower())
        list_wo_punct.append(sent)
    except:
        pass

In [220]:
len(list_wo_punct)

2002

In [221]:
# def coid19_finder(file_):
#     covid_patterns = ['covid19', 'коронавирусная', "короновирусная", "корон", "covid19", 'торсков2', 'sarscov2',"b342" ]
#     covid_list = []
#     for patt in covid_patterns:
#         for sent in file_:
#             pattern = re.sub(r'patt', 'covid', sent)
#             pattern_1 = re.compile(r'covid')
#             patt_2 = pattern_1.findall(''.join(pattern.split()))
#             try:
#                 covid_list.append(patt_2)
#             except:
#                 pass
#     c =[]
#     for i in covid_list:
#         if not i:
#             c.append(0)
#         else:
#             c.append(1)
#     return c

In [222]:
#k = coid19_finder(list_wo_punct)

In [223]:
covid_list = []

In [224]:
c =[]
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1)

In [225]:
for sent in list_wo_punct:
    pattern = re.sub(r'covid19', 'covid', sent )
    pattern = re.sub(r'коронавирусная', 'covid', pattern)
    pattern = re.sub(r'короновирусная', 'covid', pattern)
    pattern = re.sub(r'b342', 'covid', pattern)
    pattern = re.sub(r'sarscov2', 'covid', pattern)
    pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'covid')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        covid_list.append(patt_2)
    except:
        pass

In [226]:
pn_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'пневмония', 'J18', sent )
    pattern = re.sub(r'пне...ния', 'J18', pattern)
    pattern = re.sub(r'внегоспитальная', 'J18', pattern)
    pattern = re.sub(r'внебольничная', 'J18', pattern)
    #pattern = re.sub(r'sarscov2', 'covid', pattern)
    #pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'J18')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        pn_list.append(patt_2)
    except:
        pass

In [227]:
# gr_list = []
# for sent in list_wo_punct:
#     pattern = re.sub(r'тяжёлая', 'тяж', sent )
#     pattern = re.sub(r'тяжелая', 'тяж', pattern)
#     pattern = re.sub(r'тяжёлое', '', pattern)
#     pattern = re.sub(r'внебольничная', 'J18', pattern)
#     #pattern = re.sub(r'sarscov2', 'covid', pattern)
#     #pattern = re.sub(r'торсков2', 'covid', pattern)
#     pattern_1 = re.compile(r'J18')
#     patt_2 = re.search(pattern_1, pattern)
#     #patt_2 = pattern_1.findall(''.join(pattern.split()))
#     try:
#         pn_list.append(patt_2)
#     except:
#         pass

In [228]:
g = []

for i in gr_list:
    if not i:
        g.append(0)
    else:
        g.append(1)

In [229]:
dfc={}
for i in g:
    dfc[i] = dfc.get(i, 0)+1
dfc

{0: 1851, 1: 147}

In [230]:
p = []

for i in pn_list:
    if not i:
        p.append(0)
    else:
        p.append(1)

In [231]:
dfc={}
for i in p:
    dfc[i] = dfc.get(i, 0)+1
dfc

{1: 1202, 0: 800}

In [232]:
c =[]
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1)

In [233]:
dfc={}
for i in c:
    dfc[i] = dfc.get(i, 0)+1
dfc

{0: 1786, 1: 216}

In [234]:
len(c)

2002

In [235]:
list_wo_punct[19]

'поступлении внегоспитальная пневмония в нижней доле левого легкого дн 1 ст диагноз заключительный инфекция covid19 пцр от 03052020 реконвалесцент внегоспитальная пневмония в средней доле правого легкого верхней доле s3s4s5 и нижней s910 левого легкого средней степени тяжести с исходом'

In [239]:
data_dct = {'Platlets':plt_, 'Leucocytes': wbc_, 'Hemoglobin': hgb_,
           'Creatinine': crea_, 'LDG':ldh_, 'Gender':sex_, 'CRP':crp_,
            'Treatment':tr, 'Birthday': bir, 'Admission':ad,
            'Discharge':dis, 'ID':ID_, 'Diagnosis': list_wo_punct,
            'COVID-19': c, 'Pneumonia': p, 'RF': rfl, "ALT": alt_, "AST": ast_, "Procalcitonine": pct_ }

In [240]:
dataFrame = pd.DataFrame(data_dct)


In [241]:
dataFrame.head(5)

Platlets         Leucocytes       Hemoglobin Creatinine   LDG  \
0            [320]             [5.81]            [151]       None  None   
1            [444]             [7.15]            [138]       [86]  None   
2  [148, 371, 159]  [8.2, 5.76, 7.79]  [146, 164, 165]       None  None   
3             None               None             None       None  None   
4            [236]            [10.42]            [146]       None  None   

   Gender CRP                                          Treatment    Birthday  \
0  female  []   левофлоксацин . амброксол , парацетамол ,  иб...  12:09:1988   
1  female  []             триксоцеф.  азитромицин,  амброксол,    22:10:1998   
2  female  []               цефтриаксон, амброксол, плевилокс,    10:05:1963   
3  female  []                                               None  03:07:1940   
4  female  []   цефтриаксон .  амброкосл .  аспикард ,  аторв...  07:09:1957   

    Admission   Discharge      ID  \
0  23:09:2019  02:10:2019  5616.0   
1  18:01:2018  29:01:2018   430.0   
2  17:02:2020  13:03:2020  1103.0   
3  18:05:2018  01:06:2018  3338.0   
4  22:01:2019  30:01:2019   498.0   

                                           Diagnosis  COVID-19  Pneumonia RF  \
0  внегоспитальная полисегментарная пневмония в н...         0          1  0   
1  j18 пневмония без уточнения возбудителя основн...         0          1  0   
2  j18 пневмония без уточнения возбудителя основн...         0          1  0   
3                                               none         0          0  o   
4  внегоспитальная пневмония в нижней доле справа...         0          1  0   

    ALT   AST Procalcitonine  
0  None  None           None  
1  [18]  [25]           None  
2  [48]  [76]           None  
3  None  None           None  
4  None  None           None

In [156]:
dataFrame[dataFrame['ID'] == 635.0]

Platlets    Leucocytes  Hemoglobin Creatinine   LDG Gender    CRP  \
31  [364, 191]  [4.56, 8.27]  [123, 155]       None  None   male  [7,7]   

                                            Treatment    Birthday   Admission  \
31   левофлоксацин , диклофенак , амброксол , омеп...  05:06:1981  28:01:2020   

     Discharge     ID                                          Diagnosis  
31  07:02:2020  635.0  [Внегоспитальная, плевропневмония, в, н/д, спр...

In [116]:
len(dataFrame['CRP'])

1998

In [117]:
dataFrame.to_csv('data_3.csv')